In [1]:
import pandas as pd
valid_locations = [
    "TVC", "KCVL", "NCJ", "CAPE", "ERS", "QLN", "ERN", "MDU", "TEN", "SRR",
    "PGT", "CLT", "ED", "TVC", "NCJ", "ERS", "QLN", "ERN", "TEN", "MDU",
    "SRR", "PGT", "TCR", "ALLP", "KTYM", "GUV", "PUU", "KYJ", "CAR"
]

# filtered_df = df[df['from'].isin(valid_locations) & df['to'].isin(valid_locations)]
# filtered_df.to_csv('trainstartings2.csv', index=False)

In [ ]:

# import pandas as pd

# filtered_df = df[df['StationCode'].isin(valid_station_codes)]
# filtered_df.to_csv('alldetails2.csv', index=False)


In [2]:
import pandas as pd
from datetime import datetime

# Load the CSV files
trainstartings_df = pd.read_csv('trainstartings.csv')
alldetails_df = pd.read_csv('alldetails.csv')

# List of vertices of interest (valid locations)
# valid_locations = ['LOC1', 'LOC2', 'LOC3']  # Replace with your list of valid vertices

days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Helper function to convert time strings into minutes for time difference calculation
def time_to_minutes(day_str, time_str):
    days_offset = {'Sun': 6, 'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu': 3, 'Fri': 4, 'Sat': 5}
    time = datetime.strptime(time_str, "%H:%M")
    minutes = time.hour * 60 + time.minute
    total_minutes = days_offset[day_str] * 1440 + minutes
    return total_minutes

edges_data = []

for train_no in alldetails_df['trainNo'].unique():
    # Extract the journey details for this train
    train_data = alldetails_df[alldetails_df['trainNo'] == train_no]

    # Get the starting day info for this train
    train_start_days = trainstartings_df[trainstartings_df['trainNo'] == train_no]

    # Check if train_start_days is not empty
    if not train_start_days.empty:
        # Check each day of the week to find valid start days
        for i, day in enumerate(days_of_week):
            if train_start_days[day].values[0] == 1:  # Train operates on this day
                current_station = None  # Reset current station when a new train starts
                current_departure = None  # Reset current departure time when a new train starts
                
                # Loop through train data to find valid edges between vertices of interest
                for j in range(len(train_data)):
                    row = train_data.iloc[j]

                    if row['StationCode'] in valid_locations:
                        # If this is the first valid station, set it as current
                        if current_station is None:
                            current_station = row['StationCode']
                            current_departure = f"{days_of_week[(i + row['Day of Journey'] - 1) % 7][:3]} {row['Departure']}"
                        else:
                            # Create an edge between current station and the new valid station
                            departure_time = f"{days_of_week[(i + row['Day of Journey'] - 1) % 7][:3]} {row['Departure']}"
                            arrival_time = f"{days_of_week[(i + row['Day of Journey'] - 1) % 7][:3]} {row['Arrival']}"

                            # Calculate the edge weight (time difference between arrival and departure)
                            dep_minutes = time_to_minutes(current_departure.split()[0], current_departure.split()[1])
                            arr_minutes = time_to_minutes(arrival_time.split()[0], arrival_time.split()[1])
                            if arr_minutes < dep_minutes:
                                arr_minutes += 10080  # Account for day wrap-around

                            edge_weight = arr_minutes - dep_minutes

                            # Append the edge data
                            edges_data.append([
                                train_no, current_station, current_departure,
                                row['StationCode'], arrival_time, edge_weight
                            ])

                            # Update the current station and departure time
                            current_station = row['StationCode']
                            current_departure = departure_time

                # Reset the `current_station` and `current_departure` for the next train_no
                current_station = None
                current_departure = None

# Create a DataFrame with the collected edge data
edges_df = pd.DataFrame(edges_data, columns=[
    'TrainNo', 'StartingStationCode', 'DepartureTime',
    'ArrivingStationCode', 'ArrivalTime', 'EdgeWeight'
])

# Save the result to a CSV file
edges_df.to_csv('train_edges_repeated_days.csv', index=False)

print("The timetable with repeated days and valid locations has been processed and saved as 'train_edges_repeated_days.csv'.")


The timetable with repeated days and valid locations has been processed and saved as 'train_edges_repeated_days.csv'.


In [3]:
import pandas as pd
from datetime import datetime

# Specify the file path of your CSV file
file_path = 'train_edges_repeated_days.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Function to convert day and time into total minutes
def time_to_minutes(day_str, time_str):
    day_str = day_str.strip().lower()
    days_offset = {'sun': 6, 'mon': 0, 'tue': 1, 'wed': 2, 'thu': 3, 'fri': 4, 'sat': 5}
    
    time = datetime.strptime(time_str, "%H:%M")
    minutes = time.hour * 60 + time.minute
    return days_offset[day_str[:3]] * 1440 + minutes 

# Initialize total time in minutes
total_time_minutes = 0

# Loop through the rows to accumulate total time
for index in range(len(df)):
    dep_time = time_to_minutes(df.iloc[index, 2].split()[0], df.iloc[index, 2].split()[1])  # Departure time in minutes
    arr_time = time_to_minutes(df.iloc[index, 4].split()[0], df.iloc[index, 4].split()[1])  # Arrival time in minutes
    
    # Add the time difference between departure and arrival for the current row
    if arr_time < dep_time:
        total_time_minutes += 10080 - dep_time + arr_time  # You start on Sunday and reach on Monday
    else:
        total_time_minutes += arr_time - dep_time
# Convert total time to hours
total_time_hours = total_time_minutes / 60

# Print the total time in hours
print(f"Total time: {total_time_hours:.2f} hours")


Total time: 7660.77 hours
